This notebook trains and tests an SCA agent on the Laser Hockey Enviroment

In [6]:
# Imports
import numpy as np
import laserhockey.hockey_env as h_env
from importlib import reload
from SAC import Agent
import matplotlib.pyplot as plt
# from gym import wrappers

In [7]:
# Global variables
n_games = 250
load_checkpoint = True

In [8]:
# Initialization 
np.set_printoptions(suppress=True)
reload(h_env)
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_DEFENSE)

/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [9]:
# Plotting function

def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)

In [10]:
# Training of agentenv.observation_space.shape
agent = Agent(input_dims=env.observation_space.shape, 
              env=env,
              n_actions=env.action_space.shape[0])
print(agent.memory.state_memory.shape)
n_games = 250

# uncomment this line and do a mkdir tmp && mkdir video if you want to
# record video of the agent playing the game.
#env = wrappers.Monitor(env, 'tmp/video', video_callable=lambda episode_id: True, force=True)

best_score = env.reward_range[0]
score_history = []

if load_checkpoint:
    agent.load_models()
    env.render(mode='human')

for i in range(n_games):
    observation = env.reset()[0]
    done = False
    score = 0
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, _, info = env.step(action)
        score += reward
        agent.remember(observation, action, reward, observation_, done)
        if not load_checkpoint:
            agent.learn()
        observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if avg_score > best_score:
        best_score = avg_score
        if not load_checkpoint:
            agent.save_models()

    print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

if not load_checkpoint:
    x = [i+1 for i in range(n_games)]
    plot_learning_curve(x, score_history, 'SCA.png')

(1000000, 18)
.... loading models ....
episode  0 score -13.1 avg_score -13.1
episode  1 score -12.2 avg_score -12.6
episode  2 score -12.9 avg_score -12.7
episode  3 score -11.7 avg_score -12.5
episode  4 score 8.1 avg_score -8.3
episode  5 score -12.5 avg_score -9.0
episode  6 score -2.4 avg_score -8.1
episode  7 score 8.6 avg_score -6.0
episode  8 score -29.2 avg_score -8.6
episode  9 score -25.8 avg_score -10.3
episode  10 score -1.2 avg_score -9.5
episode  11 score -3.2 avg_score -8.9
episode  12 score -14.7 avg_score -9.4
episode  13 score 8.9 avg_score -8.1
episode  14 score -12.5 avg_score -8.4
episode  15 score 1.8 avg_score -7.7
episode  16 score -12.4 avg_score -8.0
episode  17 score -13.0 avg_score -8.3
episode  18 score -20.5 avg_score -8.9
episode  19 score -12.4 avg_score -9.1
episode  20 score -22.3 avg_score -9.7
episode  21 score 7.8 avg_score -8.9
episode  22 score -11.3 avg_score -9.0
episode  23 score -14.1 avg_score -9.2
episode  24 score -11.9 avg_score -9.4
epis

: 